### <p style="font-family:JetBrains Mono; font-weight:bold; letter-spacing: 2px; color:#294B8E; font-size:140%; text-align:left;padding: 0px; border-bottom: 3px solid #294B8E">Разработка рекомендательной модели на основе данных о товарах</p>

<a id="Описание-данных"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Описание данных</div>

## marketing_dealer - список дилеров

## marketing_dealerprice - результат работы парсера площадок дилеров
<ul>
<li>product_key - уникальный номер позиции;</li>
<li>price - цена;</li>
<li>product_url - адрес страницы, откуда собраны данные;</li>
<li>product_name - заголовок продаваемого товара;</li>   
<li>date - дата получения информации;</li>
<li>dealer_id - идентификатор дилера (внешний ключ к marketing_dealer)</li>
</ul>

## marketing_product - список товаров, которые производит и распространяет заказчик
<ul>
<li>article - артикул товара;</li>
<li>ean_13 - код товара (см. EAN 13);</li>
<li>name - название товара;</li>
<li>cost - стоимость;</li>
<li>min_recommended_price - рекомендованная минимальная цена;</li>
<li>recommended_price - рекомендованная цена;</li>
<li>category_id - категория товара;</li>
<li>ozon_name - названиет товара на Озоне;</li>
<li>name_1c - название товара в 1C;</li>
<li>wb_name - название товара на Wildberries;</li>
<li>ozon_article - описание для Озон;</li>
<li>wb_article - артикул для Wildberries;</li>
<li>ym_article - артикул для Яндекс.Маркета;</li>
</ul>

## marketing_productdealerkey - таблица матчинга товаров заказчика и товаров дилеров
<ul>
<li>key - внешний ключ к marketing_dealerprice</li>
<li>product_id - внешний ключ к marketing_product</li>
<li>dealer_id - внешний ключ к marketing_dealer</li>
</ul>

<div class="alert alert-block alert-info" style="background-color:#FFFAF0;
                                                 color:black;
                                                 border-color:#DEB887;
                                                 border-radius:5px;
                                                 width:50%;
                                                 margin: auto;
                                                 text-align: left;">


Необходимо разработать рекомендательную модель, которая позволяет по продукту дилера найти подходящие соответствия из рядов товаров производителя.

</div>

<a id="Загрузка-библиотек"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Загрузка библиотек</div>

In [1]:
import re
import pandas as pd
from fuzzywuzzy import fuzz, process

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Изучим наши датафреймы.

In [2]:
marketing_product = pd.read_csv('marketing_product.csv', sep = ';')
marketing_product.head()

,Unnamed: 0,id,article,ean_13,name,cost,recommended_price,category_id,ozon_name,name_1c,wb_name,ozon_article,wb_article,ym_article,wb_article_td
0,0,245,008-1,4.680008e+12,Антисептик невымываемыйPROSEPT ULTRAконцентрат...,360.00,858.0,20.0,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,Антисептик невымываемый для ответственных конс...,189522705.0,150033482.0,008-1,NaN
1,1,3,242-12,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,460.16,1075.0,NaN,NaN,Антигололед - 32 PROSEPTготовый состав / 12 кг,NaN,NaN,NaN,NaN,NaN
2,2,443,0024-06 с,4.680008e+12,"Герметик акриловый цвет сосна, ф/п 600мл",307.00,644.0,25.0,Герметик акриловый для швов для деревянных дом...,"Герметик акриловый цвет сосна, ф/п 600мл",Герметик акриловый для швов для деревянных дом...,189522735.0,150126217.0,0024-06-с,NaN
3,3,147,305-2,4.610093e+12,Кондиционер для белья с ароматом королевского...,157.73,342.0,29.0,"Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...","Кондиционер для белья ""Королевский Ирис"" Prose...",339377922.0,150032962.0,305-2,NaN
4,4,502,0024-7 б,NaN,"Герметик акриловой цвет Белый, 7 кг",NaN,NaN,NaN,NaN,NaN,NaN,189522867.0,150126216.0,0024-7-б,NaN


In [3]:
marketing_product.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 496 entries, 0 to 495
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         496 non-null    int64  
 1   id                 496 non-null    int64  
 2   article            496 non-null    object 
 3   ean_13             464 non-null    float64
 4   name               494 non-null    object 
 5   cost               491 non-null    float64
 6   recommended_price  491 non-null    float64
 7   category_id        447 non-null    float64
 8   ozon_name          458 non-null    object 
 9   name_1c            485 non-null    object 
 10  wb_name            455 non-null    object 
 11  ozon_article       365 non-null    float64
 12  wb_article         340 non-null    float64
 13  ym_article         337 non-null    object 
 14  wb_article_td      32 non-null     object 
dtypes: float64(6), int64(2), object(7)
memory usage: 58.3+ KB


In [4]:
manufacturer_data = marketing_product[['id','name']]

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Сохраним таблицу. Она необходимо, чтобы в дальнейшем не повторять этот шаг при каждом повторном вызове скрипта модели.

In [5]:
manufacturer_data.to_csv('manufacturer_data.csv', index=False)

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Загрузим объединенную таблицу.

In [6]:
manufacturer_data = pd.read_csv('manufacturer_data.csv') 

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left">Создадим рекомендательную систему.

In [7]:
def get_suitable_products(
        dealer_product: str, 
        manufactur_products: pd.Series,
        levenshtein_distance_max: int,
    ) -> list:
    '''
    A Model Explanation System
    
    return: Array of suitable manufactur products
    '''
    
    suitable_products = []
    for product in manufactur_products['name']:
        l_d = fuzz.token_sort_ratio(dealer_product, product)
        if (l_d >= levenshtein_distance_max):
            manufactur_product_id = manufactur_products[
                manufactur_products['name'] == product
            ]['id'].to_string(index=False)
            suitable_products.append({
                'id':  manufactur_product_id, 
                'product_name': product, 
                'levenshtein_distance': l_d 
            })
    return suitable_products

In [8]:
def get_solution(
        dealer_product: str,
        length: int = 10,
        manufactur_products: pd.Series = manufacturer_data,
        levenshtein_distance_max: int = 50,
    ) -> list:
    
    suitable_solution = get_suitable_products(
        dealer_product, 
        manufactur_products, 
        levenshtein_distance_max
    )
    solution = sorted(suitable_solution, key=lambda x: x['levenshtein_distance'], reverse=True)
    last_index = length if length < len(solution) else len(solution)
    
    return solution[0: last_index]
    

<a id="Загрузка-библиотек"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Тесты</div>

<div style="border-radius:10px; border:#DEB887 solid; padding: 15px; background-color: #FFFAF0; font-size:100%; text-align:left"> Проверим как отрабатывает модель при различных входных данных.

## Тест 1

### Случайное название товара дилера

In [9]:
get_solution('Средство для удаления ленты  клейкой ')

[{'id': '129',
  'product_name': 'Средство для удаления клейкой ленты, клея, наклеекDuty Universalготовый состав / 2 л',
  'levenshtein_distance': 61},
 {'id': '1',
  'product_name': 'Средство для удаления наклеек Duty Universal / 210мл',
  'levenshtein_distance': 61},
 {'id': '128',
  'product_name': 'Средство для удаления клейкой ленты, клея, наклеекDuty Universalготовый состав / 400мл',
  'levenshtein_distance': 60},
 {'id': '408',
  'product_name': 'Средство для снятия обоев спрей / 1 л',
  'levenshtein_distance': 51},
 {'id': '122',
  'product_name': 'Средство для удаления гипсовой пылиDuty Whiteконцентрат 1:10-1:100 / 5 л  ',
  'levenshtein_distance': 50},
 {'id': '121',
  'product_name': 'Средство для удаления гипсовой пылиDuty Whiteконцентрат 1:10-1:100 / 1 л  ',
  'levenshtein_distance': 50}]

## Тест 2

### Случайное название товара дилера

In [10]:
get_solution('Антисептик для влажной древесины PROSEPT BiO, концентрат 1:19 20 л')

[{'id': '267',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19 / 20 л',
  'levenshtein_distance': 72},
 {'id': '268',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19  / 30 л',
  'levenshtein_distance': 70},
 {'id': '266',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19 / 5 л',
  'levenshtein_distance': 69},
 {'id': '265',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19  / 1 л',
  'levenshtein_distance': 68},
 {'id': '498',
  'product_name': 'Антисептик ULTRA, концентрат, 1 л, 2 шт',
  'levenshtein_distance': 57},
 {'id': '500',
  'product_name': 'Антисептик ULTRA MAX, концентрат, 1 кг, 2 шт',
  'levenshtein_distance': 57},
 {'id': '501',
  'product_name': 'Антисептик\xa0 SAUNA, концентрат, 1 л, 2 шт',
  'levenshtein_distance': 57},
 {'id': '255',
  'product_name': 'Антисептик трудновымываемый для наружных работ PROSEPT EXTERIORконцентрат 1:19 / 20 л',
  'levenshtein_d

## Тест 3

### Случайное название товара дилера

In [11]:
get_solution('Удалитель мха готовый состав 1 л PROSEPT')

[{'id': '209',
  'product_name': 'Удалитель мха готовый состав / 1 л ',
  'levenshtein_distance': 89},
 {'id': '210',
  'product_name': 'Удалитель мха готовый состав / 5 л ',
  'levenshtein_distance': 86},
 {'id': '484',
  'product_name': 'Набор Удалитель мха, готовый состав, 1 л, 2 шт',
  'levenshtein_distance': 80},
 {'id': '403',
  'product_name': 'Удалитель ржавчины RUST REMOVER готовый состав / 0,5 л',
  'levenshtein_distance': 74},
 {'id': '324',
  'product_name': 'Лак для камня PROSEPTготовый состав / 2 л',
  'levenshtein_distance': 68},
 {'id': '218',
  'product_name': 'Лак яхтный PROSEPT, глянцевыйготовый состав / 2 л ',
  'levenshtein_distance': 67},
 {'id': '390',
  'product_name': 'Удалитель плесени FUNGI CLEAN готовый состав / 0,5 л',
  'levenshtein_distance': 67},
 {'id': '323',
  'product_name': 'Лак для камня PROSEPTготовый состав / 0,9 л',
  'levenshtein_distance': 67},
 {'id': '328',
  'product_name': 'Лак яхтный PROSEPT, полуматовый,готовый состав / 0,9 л',
  'levens

## Тест 4

### C заданной длинной рекомендованных товаров 5 (по умолчанию 10)

In [12]:
get_solution('Антисептик для влажной древесины PROSEPT BiO, концентрат 1:19 20 л', length=5)

[{'id': '267',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19 / 20 л',
  'levenshtein_distance': 72},
 {'id': '268',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19  / 30 л',
  'levenshtein_distance': 70},
 {'id': '266',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19 / 5 л',
  'levenshtein_distance': 69},
 {'id': '265',
  'product_name': 'Антисептик для влажной древесиныPROSEPT BiOконцентрат 1:19  / 1 л',
  'levenshtein_distance': 68},
 {'id': '498',
  'product_name': 'Антисептик ULTRA, концентрат, 1 л, 2 шт',
  'levenshtein_distance': 57}]

In [34]:
get_solution('Просепт Professional Splash Soft средство для мытья посуды в посудомоечной машине (5 л)')

[{'id': '99',
  'product_name': 'Средство для мытья посуды в посудомоечной машине. Для жесткой водыCooky Splash Softконцентрат 1:200-1:2000 / 20 л',
  'levenshtein_distance': 66},
 {'id': '98',
  'product_name': 'Средство для мытья посуды в посудомоечной машине. Для жесткой водыCooky Splash Softконцентрат 1:200-1:2000 / 10 л',
  'levenshtein_distance': 66},
 {'id': '97',
  'product_name': 'Средство для мытья посуды в посудомоечной машине. Для жесткой водыCooky Splash Softконцентрат 1:200-1:2000 / 5 л',
  'levenshtein_distance': 66},
 {'id': '100',
  'product_name': 'Средство для мытья посуды в посудомоечной машине. Для жесткой водыCooky Splash Hard концентрат 1:200-1:2000 / 5 л',
  'levenshtein_distance': 63},
 {'id': '506',
  'product_name': 'Набор для посудомоечной посуды PROSEPT, 3 средства',
  'levenshtein_distance': 61},
 {'id': '102',
  'product_name': 'Средство для мытья посуды в посудомоечной машине. Для жесткой водыCooky Splash Hard концентрат 1:200-1:2000 / 20 л',
  'levensht

<a id="Загрузка-библиотек"></a>
<div style="padding:20px;
            color:white;
            margin:10;
            font-size:170%;
            text-align:left;
            display:fill;
            border-radius:5px;
            background-color:#294B8E;
            overflow:hidden;
            font-weight:700"><span style='color:#CDA63A'>|</span> Выводы</div>

***Подготовлена*** рекомендательная модель для сопоставления товара дилера с товарами производителя.
Функция `get_solution()`принимает строковую переменную товара дилера и возвращает список рекомендованных товаров продавца.

***Фичи:***

    - ✅ работает для случайного набора слов в названии товара дилера;    
    - ✅ отрабатывает с сырыми данными парсинга;
    - ✅ позволяет работать с заданным значением длины возвращаемого массива товаров.
